In [19]:
#Constants
DATA_PATH = "/Users/Hashem/Link TSP/Project1/data/raw/survey_results_public.csv"
EXPORT_PATH = 3

REPLACE_DICT ={
    'YearCodePro' : {'Less than 1 year' : 0, 'More than 50 years' : 51},
    'YearCode' : {'Less than 1 year' : 0, 'More than 50 years' : 51},
    'Ahe1stcode' : {'Older than 85' : 86, 'Younger than 5 years' : 4},
}

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 10000

In [ ]:
def split_answers(data_series, delimiter=';'):
    """
    Split multiple answer is a signal string
    to a list of single each represnting a single answer

    parameters:
    data_series (pd.Series): String series with answers
    delimiter (string): Another decimal integer
                        Defaults to ";"
    
    Returns: (pd.Series): If column contains
    """
    #sub function
    def is_splittable(pd_series, delimiter):
        """ Check if results multiple should be sp;itted - Returns boolean """
        return pd_series.str.contains(delimiter)

    def split_answer(pd_series, delimiter):
        """ Function to split signle answer"""
        return pd_series.str.split(delimiter)

    #-------------------------------------------
    #check if multiple answer exist - if name: return original
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    #Else, split each value to a list
    modified_series = split_answer(data_series, delimiter)

    #replace NAs with empty lists
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])



In [21]:
raw_df= pd.read_csv(DATA_PATH)
raw_df

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64456,64858,NaN,Yes,NaN,16,NaN,NaN,NaN,United States,NaN,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,NaN,NaN,10,Less than 1 year
64457,64867,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Morocco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64458,64898,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Viet Nam,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64459,64925,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Poland,NaN,...,NaN,NaN,NaN,NaN,Angular;Angular.js;React.js,NaN,NaN,NaN,NaN,NaN


In [22]:
raw_df.shape

(64461, 61)

In [23]:
#Display random answer
#Observations: Multiple answers need to be splitted
#Reference to the schema needed to understannd
raw_df.sample(1).iloc[0]

Respondent                                                                  11366
MainBranch                                         I am a developer by profession
Hobbyist                                                                      Yes
Age                                                                          30.0
Age1stCode                                                                     12
CompFreq                                                                  Monthly
CompTotal                                                                 44000.0
ConvertedComp                                                            133644.0
Country                                                                    Poland
CurrencyDesc                                                         Polish zloty
CurrencySymbol                                                                PLN
DatabaseDesireNextYear                         Elasticsearch;Microsoft SQL Server
DatabaseWorkedWi

In [24]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Respondent                    64461 non-null  int64  
 1   MainBranch                    64162 non-null  object 
 2   Hobbyist                      64416 non-null  object 
 3   Age                           45446 non-null  float64
 4   Age1stCode                    57900 non-null  object 
 5   CompFreq                      40069 non-null  object 
 6   CompTotal                     34826 non-null  float64
 7   ConvertedComp                 34756 non-null  float64
 8   Country                       64072 non-null  object 
 9   CurrencyDesc                  45472 non-null  object 
 10  CurrencySymbol                45472 non-null  object 
 11  DatabaseDesireNextYear        44070 non-null  object 
 12  DatabaseWorkedWith            49537 non-null  object 
 13  D

In [25]:
raw_df.describe()

,Respondent,Age,CompTotal,ConvertedComp,WorkWeekHrs
count,64461.000000,45446.000000,3.482600e+04,3.475600e+04,41151.000000
mean,32554.079738,30.834111,3.190464e+242,1.037561e+05,40.782174
std,18967.442360,9.585392,inf,2.268853e+05,17.816383
min,1.000000,1.000000,0.000000e+00,0.000000e+00,1.000000
25%,16116.000000,24.000000,2.000000e+04,2.464800e+04,40.000000
50%,32231.000000,29.000000,6.300000e+04,5.404900e+04,40.000000
75%,49142.000000,35.000000,1.250000e+05,9.500000e+04,44.000000
max,65639.000000,279.000000,1.111111e+247,2.000000e+06,475.000000


In [26]:
questionable_cols=['YearsCodePro', 'YearsCode', 'Age1stCode']

In [27]:
for col in questionable_cols:
    print(col)
    print(raw_df[col].unique().tolist())
    print('---------------')
    print()

YearsCodePro
['27', '4', nan, '8', '13', '2', '7', '20', '1', '23', '3', '12', '17', '18', '10', '14', '29', '6', '28', '9', '15', '11', '16', '25', 'Less than 1 year', '5', '21', '19', '35', '24', '32', '22', '30', '38', '26', '40', '33', '31', 'More than 50 years', '34', '36', '39', '37', '41', '45', '47', '42', '46', '50', '43', '44', '48', '49']
---------------

YearsCode
['36', '7', '4', '15', '6', '17', '8', '10', '35', '5', '37', '19', '9', '22', '30', '23', '20', '2', 'Less than 1 year', '3', '13', '25', '16', '43', '11', '38', '33', nan, '24', '21', '12', '40', '27', '50', '46', '14', '18', '28', '32', '44', '26', '42', '31', '34', '29', '1', '39', '41', '45', 'More than 50 years', '47', '49', '48']
---------------

Age1stCode
['13', '19', '15', '18', '16', '14', '12', '20', '42', '8', '25', '22', '30', '17', '21', '10', '46', '9', '7', '11', '6', nan, '31', '29', '5', 'Younger than 5 years', '28', '38', '23', '27', '41', '24', '53', '26', '35', '32', '40', '33', '36', '54', '

In [28]:
raw_df.nunique()

Respondent                      64461
MainBranch                          5
Hobbyist                            2
Age                               110
Age1stCode                         63
CompFreq                            3
CompTotal                        3003
ConvertedComp                    6958
Country                           183
CurrencyDesc                      142
CurrencySymbol                    141
DatabaseDesireNextYear           3193
DatabaseWorkedWith               2808
DevType                          8269
EdLevel                             9
Employment                          7
Ethnicity                         208
Gender                              7
JobFactors                        230
JobSat                              5
JobSeek                             3
LanguageDesireNextYear          16243
LanguageWorkedWith              14256
MiscTechDesireNextYear           5216
MiscTechWorkedWith               2730
NEWCollabToolsDesireNextYear     1277
NEWCollabToo